In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 64
display_step = 20

In [4]:
# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.8 # Dropout, probability to keep units

In [5]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [6]:
# Create custom model
def conv2d(name, l_input, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(l_input, w, strides=[1, 1, 1, 1], padding="SAME"),b), name=name)

def max_pool(name, l_input, k):
    return tf.nn.max_pool(l_input, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding="SAME", name=name)


In [7]:
def dnn(_X, _weights, _biases, _dropout):
    _X = tf.nn.dropout(_X, dropout)
    d1 = tf.nn.relu(tf.nn.bias_add(tf.matmul(_X,_weights["wd1"]), _biases["bd1"]), name = "d1")
    d2_X = tf.nn.dropout(d1, _dropout)
    d2 = tf.nn.relu(tf.nn.bias_add(tf.matmul(d2_X, _weights["wd2"]), _biases["bd2"]), name = "d2")
    
    dout = tf.nn.dropout(d2, _dropout)
    out = tf.matmul(dout, _weights["out"]) + _biases["out"]
    
    return out

In [10]:
#Store weights & biases
weights = {
    "wd1" : tf.Variable(tf.random_normal([784,600], stddev = 0.01)),
    "wd2" : tf.Variable(tf.random_normal([600,480], stddev = 0.01)),
    "out" : tf.Variable(tf.random_normal([480,10]))
}

biases = {
    "bd1" : tf.Variable(tf.random_normal([600])),
    "bd2" : tf.Variable(tf.random_normal([480])),
    "out" : tf.Variable(tf.random_normal([10]))
}

In [11]:
#Construct the model
prediction = dnn(x, weights, biases, keep_prob)

In [12]:
#Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

In [13]:
#Evaluate the model
correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [14]:
#Initializing the variables
init = tf.global_variables_initializer()

In [15]:
#Launch the graph
with tf.Session() as session:
    session.run(init)
    
    step = 1
    while step * batch_size < training_iters:
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        #Fit training
        session.run(optimizer, feed_dict = {x: batch_xs, y: batch_ys, keep_prob: dropout})
        if step % display_step == 0:
            #Calculate the accuracy and loss
            acc = session.run(accuracy, feed_dict = {x: batch_xs, y: batch_ys, keep_prob: 1})
            loss = session.run(cost, feed_dict = {x: batch_xs, y: batch_ys, keep_prob: 1})
            
            print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc)
        step += 1

print "Optimization Finished"
#Calculate accuracy for the first 256 mnist images
print "Testing accuracy: ", session.run(accuracy, feed_dict = {x: mnist.test.images[:256], y: mnist.test.labels[:256], keep_prob: 1})

Iter 1280, Minibatch Loss= 2.112351, Training Accuracy= 0.51562
Iter 2560, Minibatch Loss= 0.944466, Training Accuracy= 0.71875
Iter 3840, Minibatch Loss= 1.093474, Training Accuracy= 0.82812
Iter 5120, Minibatch Loss= 0.558806, Training Accuracy= 0.85938
Iter 6400, Minibatch Loss= 0.589973, Training Accuracy= 0.84375
Iter 7680, Minibatch Loss= 0.593365, Training Accuracy= 0.82812
Iter 8960, Minibatch Loss= 0.355986, Training Accuracy= 0.89062
Iter 10240, Minibatch Loss= 0.374402, Training Accuracy= 0.89062
Iter 11520, Minibatch Loss= 0.475308, Training Accuracy= 0.87500
Iter 12800, Minibatch Loss= 0.433041, Training Accuracy= 0.85938
Iter 14080, Minibatch Loss= 0.364004, Training Accuracy= 0.87500
Iter 15360, Minibatch Loss= 0.141492, Training Accuracy= 0.95312
Iter 16640, Minibatch Loss= 0.539061, Training Accuracy= 0.90625
Iter 17920, Minibatch Loss= 0.617608, Training Accuracy= 0.87500
Iter 19200, Minibatch Loss= 0.316130, Training Accuracy= 0.90625
Iter 20480, Minibatch Loss= 0.27

Iter 162560, Minibatch Loss= 0.169995, Training Accuracy= 0.93750
Iter 163840, Minibatch Loss= 0.079353, Training Accuracy= 0.95312
Iter 165120, Minibatch Loss= 0.206304, Training Accuracy= 0.96875
Iter 166400, Minibatch Loss= 0.272410, Training Accuracy= 0.90625
Iter 167680, Minibatch Loss= 0.081220, Training Accuracy= 0.98438
Iter 168960, Minibatch Loss= 0.172516, Training Accuracy= 0.93750
Iter 170240, Minibatch Loss= 0.053157, Training Accuracy= 0.96875
Iter 171520, Minibatch Loss= 0.021406, Training Accuracy= 1.00000
Iter 172800, Minibatch Loss= 0.038169, Training Accuracy= 0.98438
Iter 174080, Minibatch Loss= 0.034660, Training Accuracy= 1.00000
Iter 175360, Minibatch Loss= 0.057304, Training Accuracy= 0.98438
Iter 176640, Minibatch Loss= 0.063920, Training Accuracy= 0.96875
Iter 177920, Minibatch Loss= 0.036979, Training Accuracy= 0.98438
Iter 179200, Minibatch Loss= 0.012705, Training Accuracy= 1.00000
Iter 180480, Minibatch Loss= 0.239505, Training Accuracy= 0.95312
Iter 18176

RuntimeError: Attempted to use a closed Session.